In [ ]:
%matplotlib qt

In [ ]:
import random
import time

from mne.time_frequency import psd_multitaper
from mne import find_events, write_events
from mne.io import RawArray

import numpy as np

from pylsl import local_clock

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import rteeg

In [1]:
import time
from mne import find_events
import rteeg
rt = rteeg.Stream()
rt.connect(eeg=True, markers=True)

Searching for EEG stream ... 
Searching for Markers stream ... 
Connected to Markers stream. 
Connected to EEG stream. 


In [2]:
rt.eeg_latency()

0.26665426299587125

In [9]:
global_times = [0]
n_samples = [0]
def debug_func(global_times_, n_samples_):
    n_samples_1 = len(rt._eeg_data)
    delta_n_samples = n_samples_1 - n_samples_[-1]
    n_samples_.append(n_samples_1)

    
    if global_times_:
        t1 = time.time()
        t_delta_main = t1 - global_times_[-1]
        global_times_.append(t1)
    else:
        global_times_.append(time.time())
        t_delta_main = 0.
    
    
    true_last_event = rt._marker_data[-1][0]
    latency = rt.eeg_latency()
    # time.sleep(latency)
    last_event = find_events(rt.make_raw(5))[-1, 2]
    
    out = ("{:.2f}\t{:.2f}\t\t{}\t\t{}\t\t{}"
          "".format(latency, t_delta_main, last_event, true_last_event, delta_n_samples))
    print out

In [10]:
print "Latency\tDelta-time\tlast event\ttrue last\tn_samples"
loop = rteeg.LoopAnalysis(rt, 3, debug_func, (global_times, n_samples),
                          show_window=False)

Latency	Delta-time	last event	true last	n_samples
1.89	1482275022.92		557		558		233615
2.68	3.80		558		559		1502
1.71	2.03		560		560		1502
1.04	2.34		561		562		1500
0.69	2.64		563		563		1500
1.24	3.55		564		565		1500


In [8]:
loop.stop()

Loop of analysis stopped.


In [ ]:
predictions = []
these_times = []

def analysis_func(list_, global_times):
    output_base = """
    <head>
        <style>
        table {{font-size: 30px;}}
        th, td {{
            background-color: white;
            text-align: center;
            padding: 20px;
        }}

        </style>
    </head>
    <body>

        <table align="center">
            <tr>
                <th>Prediction</th>
                <th>EEG samples (N)</th>
                <th>Time since last prediction</th>
                <th>Latency</th>
                <th>i</th>
            </tr>
            <tr>
                <td style="background-color:{color}"><b>{prediction}</b></td>
                <td>{n_samples}</td>
                <td>{t_delta_main:.4f}</td>
                <td>{latency:.2f}</td>
                <td>{i}</td>
            </tr>
        </table>

    </body>
    """
    n_samples = len(stream._eeg_data)
    event_id = {
        1: ["Class 1", "green"],
        2: ["Class 2", "red"],
    }

    y_pred = random.choice(event_id.keys())
    list_.append(y_pred)
    
    if global_times:
        t1 = time.time()
        t_delta_main = t1 - global_times[-1]
        global_times.append(t1)
    else:
        global_times.append(time.time())
        t_delta_main = 0.
        
    out = output_base.format(prediction=event_id[y_pred][0],
                             color=event_id[y_pred][1],
                             n_samples=n_samples,
                             latency=stream.eeg_latency(),
                             t_delta_main=t_delta_main,
                             i=len(list_))
    t_delta_main = None
    
    return out

loop = rteeg.LoopAnalysis(stream, 1, analysis_func, (predictions, these_times), 
                          show_window=True)

In [ ]:
from threading import Thread, Event
from pylsl import local_clock

class ContinuousResolver(object):
    """Class to continously update the latency
    of the EEG recording."""
    def __init__(self, stream):
        
        self.stream = stream
        self.latency = 0.
        self.i = 0
        self.stopped = Event()
        
        # Raise error if EEG stream not started.
        self.stream._check_if_stream_active('EEG')
        
        thread = Thread(target=self._resolve, name="Latency-resolver")
        thread.daemon = True
        thread.start()
        
    def _resolve(self):
        # Get measure of latency every 10 ms.
        while not self.stopped.wait(0.01):
            self.latency = local_clock() - self.stream._eeg_data[-1][-1]
            if self.latency > 3.:
                self.i += 1
            
    def stop(self):
        self.stopped.set()

In [ ]:
resolve = ContinuousResolver(rt)

In [ ]:
print resolve.latency
print(local_clock() - rt._eeg_data[-1][-1])

In [ ]:
print rt._eeg_data[-1][-1]

In [ ]:
resolve.i

In [ ]:
len(rt._eeg_data)

In [ ]:
rt.recording_duration()
print rt.eeg_latency()

In [ ]:
for __ in range(10):
    print rt.eeg_latency()
    time.sleep(2)

In [ ]:
data = stream._get_raw_eeg_data()
events = stream.make_events(data)
print events.shape
data[-1,:] = 0
raw = RawArray(data, stream.info)
raw.add_events(events)

In [ ]:
events = find_events(raw, shortest_event=0)

In [ ]:
write_events("rteeg_test_Dec8-events.fif", events)

In [ ]:
raw.info['events'] = dict()

In [ ]:
find_events(raw, shortest_event=0).shape

In [ ]:
raw.save('rteeg_test_Dec8-raw.fif', overwrite=True)

In [ ]:
find_events(raw, shortest_event=0)

In [ ]:
stream = rteeg.Stream()

In [ ]:
stream.connect(eeg=True, markers=False, eeg_montage='Enobio32')

In [ ]:
stream.recording_duration()

In [ ]:
raw = stream.make_raw()
# raw.plot()
print find_events(raw)

In [ ]:
stream.fit_ica(20)

In [ ]:
epochs = stream.make_epochs()

In [ ]:
raw = stream.make_raw(10)
events = find_events(raw)
print events
raw.plot(events=events)

In [ ]:
predictions = []
these_times = []

def analysis_func(list_, global_times):
    output_base = """
    <head>
        <style>
        table {{font-size: 30px;}}
        th, td {{
            background-color: white;
            text-align: center;
            padding: 20px;
        }}

        </style>
    </head>
    <body>

        <table align="center">
            <tr>
                <th>Prediction</th>
                <th>EEG samples (N)</th>
                <th>Time since last prediction</th>
                <th>Latency</th>
                <th>i</th>
            </tr>
            <tr>
                <td style="background-color:{color}"><b>{prediction}</b></td>
                <td>{n_samples}</td>
                <td>{t_delta_main:.4f}</td>
                <td>{latency:.2f}</td>
                <td>{i}</td>
            </tr>
        </table>

    </body>
    """
    n_samples = len(stream._eeg_data)
    event_id = {
        1: ["Class 1", "green"],
        2: ["Class 2", "red"],
    }

    y_pred = random.choice(event_id.keys())
    list_.append(y_pred)

#     t0 = time.time()
#     raw = stream.make_raw(10)
#     raw.filter(1., 40)
#     psd, freqs= psd_multitaper(raw, fmin=1., fmax=7.0)
#     psd_scaled = StandardScaler().fit_transform(psd)
#     pca = PCA(n_components=15).fit_transform(psd_scaled)
#     t_delta = time.time() - t0
    
    if global_times:
        t1 = time.time()
        t_delta_main = t1 - global_times[-1]
        global_times.append(t1)
    else:
        global_times.append(time.time())
        t_delta_main = 0.
        
    out = output_base.format(prediction=event_id[y_pred][0],
                             color=event_id[y_pred][1],
                             n_samples=n_samples,
                             latency=stream.eeg_latency(),
                             t_delta_main=t_delta_main,
                             i=len(list_))
    t_delta_main = None
    
    return out

loop = rteeg.LoopAnalysis(stream, 1, analysis_func, (predictions, these_times), 
                          show_window=True)